In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
# import tushare as ts
# import wrds
import talib as ta
import datetime, time

from scipy import stats
from dateutil.relativedelta import *
from pandas.tseries.offsets import *

In [3]:
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'
plt.style.use('ggplot')

In [4]:
crsp = pd.read_csv('data/return.zip', 
                   parse_dates=['date'], infer_datetime_format=True)
crsp[['PERMNO', 'PERMCO']] = crsp[['PERMNO', 'PERMCO']].astype(int)
crsp_name = pd.read_csv('data/crsp_name.csv', 
                   parse_dates=['namedt','nameendt'], infer_datetime_format=True)
crsp_name = crsp_name.drop('Unnamed: 0', axis=1)
crsp_name[['permno', 'shrcd', 'exchcd']] = \
crsp_name[['permno', 'shrcd', 'exchcd']].astype(int)

crsp_name['namedt'] = pd.to_datetime(crsp_name['namedt'])
crsp_name['nameendt'] = pd.to_datetime(crsp_name['nameendt'])
# if nameendt is missing then set to today date
crsp_name['nameendt'] = crsp_name['nameendt'].fillna(pd.to_datetime('today'))
crsp_merge = pd.merge(crsp, crsp_name, 'inner', left_on='PERMNO', right_on='permno')
crsp_merge = crsp_merge[(crsp_merge.date <= crsp_merge.nameendt) &
                       (crsp_merge.date >= crsp_merge.namedt)]
crsp_merge.RET = crsp_merge.RET.replace(['B', 'C'], [np.nan, np.nan]).astype(np.float64)
crsp_merge.RETX = crsp_merge.RETX.replace(['B', 'C'], [np.nan, np.nan]).astype(np.float64)

crsp_merge.RET = crsp_merge.RET.fillna(0)
crsp_merge.RETX = crsp_merge.RETX.fillna(0)

crsp_merge = crsp_merge.drop(['permno', 'namedt', 'nameendt'], axis=1)

crsp_merge.columns = [i.lower() for i in crsp_merge.columns]

E:\ProgramData\Anaconda3\envs\quant\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [9]:
crsp_merge = crsp_merge[['permno', 'date', 'ticker', 'comnam',  
                         'prc', 'ret', 'shrout', 'retx']]

### calculate the mv, ret+1, retx+1

In [10]:
# calculate the mv, ret+1, retx+1
crsp1 = crsp_merge.assign(mv = crsp_merge.prc.abs() * crsp_merge.shrout,
                         ret_p1 = crsp_merge.ret+1,
                         retx_p1 = crsp_merge.retx+1)

In [12]:
# resample : use calendar year
def normal_resample(df):
    res = df.resample('Y').apply({'ret_p1':'cumprod', 'retx_p1':'cumprod'})
    res = pd.concat([res, df.mv], axis=1)
    res = res.resample('Y').last()
    return res

In [15]:
# test_code_list = crsp1.permno.unique().tolist()[-5:]
# test_data = crsp1.set_index('permno').iloc[test_code_list]
# test_data = test_data.reset_index().set_index('date')
# test_data.groupby('permno').apply(normal_resample)

,,ret_p1,retx_p1,mv
permno,date,,,
10234,1974-12-31,1.000000,1.000000,NaN
17778,1981-12-31,1.119318,1.119318,485605.000
19271,2000-12-31,1.072917,1.072917,137801.125


In [16]:
tic = time.perf_counter()
crsp2 = crsp1.set_index('date').groupby('permno').apply(normal_resample)
toc = time.perf_counter()
print(toc-tic)

210.2191183


In [17]:
crsp2.head()

ret_p1   retx_p1          mv
permno date                                      
10000  1986-12-31  0.117857  0.117857  1981.56609
       1987-12-31  0.424243  0.424243   851.59375
10001  1986-12-31  1.217369  1.142857  6937.00000
       1987-12-31  0.898725  0.839285  5828.00000
       1988-12-31  1.163160  1.085106  6362.25000

example : America Online

In [24]:
code = 77418 # America Online的permno
sample_data = crsp2.loc[(code, slice(None))]

In [29]:
# calculate the EXF
sample_data = sample_data.assign(
    exf = sample_data.mv - sample_data.mv.shift(1) * sample_data.retx_p1
)

In [30]:
sample_data

,ret_p1,retx_p1,mv,exf
date,,,,
1992-12-31,1.918033,1.918033,1.678072e+05,NaN
1993-12-31,2.000764,2.000001,4.111965e+05,7.558184e+04
1994-12-31,1.914528,1.914528,9.291520e+05,1.419047e+05
1995-12-31,2.678568,2.678568,3.246112e+06,7.573156e+05
1996-12-31,0.886665,0.886665,3.104486e+06,2.262706e+05
1997-12-31,2.721810,2.721810,9.313355e+06,8.635347e+05
1998-12-31,6.934765,6.934712,7.106990e+07,6.484468e+06
1999-12-31,1.956485,1.956485,1.696175e+08,3.057033e+07
2000-12-31,0.458649,0.458649,8.087910e+07,3.084190e+06


In [36]:
mv_2000 = sample_data['2000'].mv.iloc[0]
mv_2001 = sample_data['2001'].mv.iloc[0]
rx_2001 = sample_data['2001'].retx_p1.iloc[0] - 1
exf_2001 = sample_data['2001'].exf.iloc[0]
print(mv_2000, mv_2001, rx_2001, exf_2001)

80879097.6 136599880.8 -0.0775866884424 61995924.547


通过上表,我们可以观察到$MV_{2000}$为80879, $MV_{2001}$为136600(四舍五入),

$rx_{2001}$为-7.76%, $EXF_{2001}$为61996.